In [1]:
import torch
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig
from peft import PeftConfig, PeftModel
from datasets import load_dataset, DatasetDict
from peft import prepare_model_for_kbit_training

In [7]:
from huggingface_hub import login
login()

In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "Trelis/Llama-2-7b-chat-hf-sharded-bf16" ## "Trelis/Llama-2-7b-chat-hf-sharded-bf16" is an alternative if you don't have access via Meta on HuggingFace
# model_id = "meta-llama/Llama-2-13b-chat-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

config.json:   0%|          | 0.00/658 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00007.bin:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00007.bin:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

pytorch_model-00003-of-00007.bin:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

...

pytorch_model-00007-of-00007.bin:   0%|          | 0.00/1.66G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

/opt/miniconda/envs/llm/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/197 [00:00<?, ?B/s]

/opt/miniconda/envs/llm/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/miniconda/envs/llm/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [13]:
tokenizer = "BiniyamAjaw/amharic_tokenizer"
tokenizer = AutoTokenizer.from_pretrained(tokenizer)

tokenizer.tokenize("አሁንእንደምንችልእንደምንችልእንደምንችል")

['አሁን', 'እንደምንችል', 'እንደምንችል', 'እንደምንችል']

In [14]:
from os import path
dataset_dict = load_dataset('text', data_files='/home/biniyam_ajaw/finetuning/merged/merged.txt')

dataset = dataset_dict['train']

train_test = dataset.train_test_split(test_size=0.2)
dataset = train_test


context_len = 1024
def tokenize(element):
    return tokenizer(
        element['text'],
        truncation=True,
        padding = True,
        max_length = context_len,
        return_overflowing_tokens=True
    )
    
tokenized_datasets = dataset.map(
    tokenize, batched=True, remove_columns=dataset['train'].column_names
)
tokenized_datasets

Map:   0%|          | 0/324979 [00:00<?, ? examples/s]

Map:   0%|          | 0/81245 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'overflow_to_sample_mapping'],
        num_rows: 325044
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'overflow_to_sample_mapping'],
        num_rows: 81263
    })
})

In [ ]:
#Resize token embeddings

model.resize_token_embeddings(
    new_num_tokens=len(tokenizer),
    pad_to_multiple_of=64)   # phi2 default is 64, see configuration_phi.py
model.config.eos_token_id = tokenizer.eos_token_id

In [17]:

dataset.push_to_hub("amharic_dataset")

HfHubHTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/repos/create (Request ID: Root=1-65bb28e1-28ca3f834252505d367d6e6b;20c3b253-9632-4cba-9a43-317efb6f16d4)

Invalid username or password.

In [18]:
def collate(elements):

    # Extract input_ids from each element and find the maximum length among them
    tokens = [e["input_ids"] for e in elements]
    tokens_maxlen = max([len(t) for t in tokens])

    for i, sample in enumerate(elements):
        input_ids = sample["input_ids"]
        labels = sample["labels"]
        attention_mask = sample["attention_mask"]

        # Calculate the padding length required to match the maximum token length
        pad_len = tokens_maxlen-len(input_ids)

        # Pad 'input_ids' with the pad token ID, 'labels' with IGNORE_INDEX, and 'attention_mask' with 0
        input_ids.extend( pad_len * [tokenizer.pad_token_id] )
        labels.extend( pad_len * [IGNORE_INDEX] )
        attention_mask.extend( pad_len * [0] )

    # create and return batch with all the data in elements
    batch={
        "input_ids": torch.tensor( [e["input_ids"] for e in elements] ),
        "labels": torch.tensor( [e["labels"] for e in elements] ),
        "attention_mask": torch.tensor( [e["attention_mask"] for e in elements] ),
    }
    return batch

In [19]:
num_tokens_train = len(tokenized_datasets['train'])
num_tokens_train

325044

In [20]:
from peft import prepare_model_for_kbit_training
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)


def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [37]:
from peft import LoraConfig, get_peft_model

# Configure LoRA settings for fine-tuning
lora_config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules = ["self_attn.q_proj", "self_attn.k_proj", "self_attn.v_proj", "self_attn.o_proj"],
    lora_dropout=0.1,
    bias="none",
    modules_to_save = ["lm_head", "embed_tokens"],   # because we added new tokens
    task_type="CAUSAL_LM"
)

# Add adapters to model
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing = False)
model = get_peft_model(model, lora_config)
model.config.use_cache = False

In [ ]:
#Resize token embeddings

model.resize_token_embeddings(
    new_num_tokens=len(tokenizer),
    pad_to_multiple_of=64)   # phi2 default is 64, see configuration_phi.py
model.config.eos_token_id = tokenizer.eos_token_id

In [25]:
bs=1         # batch size
ga_steps=16  # gradient acc. steps
epochs=20
lr=0.00002

In [26]:
# steps_per_epoch=len(tokenized_datasets["train"])//(bs*ga_steps)

# args = TrainingArguments(
#     output_dir="out",
#     per_device_train_batch_size=bs,
#     per_device_eval_batch_size=16,
#     evaluation_strategy="steps",
#     logging_steps=1,
#     eval_steps=steps_per_epoch//2,      # eval twice per epoch
#     save_steps=steps_per_epoch,         # save once per epoch
#     gradient_accumulation_steps=ga_steps,
#     num_train_epochs=epochs,
#     lr_scheduler_type="constant",
#     optim="paged_adamw_32bit",      # val_loss will go NaN with paged_adamw_8bit
#     learning_rate=lr,
#     group_by_length=False,
#     bf16=True,
#     ddp_find_unused_parameters=False,
# )

In [28]:
# trainer = Trainer(
#     model=model,
#     tokenizer=tokenizer,
#     args=args,
#     data_collator=collate,
#     train_dataset=tokenized_datasets["train"],
#     eval_dataset=tokenized_datasets["test"],
# )



In [35]:
from transformers import DataCollatorForLanguageModeling
tokenizer.pad_token = '[PAD]'
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

out = data_collator([tokenized_datasets["train"][i] for i in range(5)])
for key in out:
    print(f"{key} shape: {out[key].shape}")

input_ids shape: torch.Size([5, 732])
token_type_ids shape: torch.Size([5, 732])
attention_mask shape: torch.Size([5, 732])
overflow_to_sample_mapping shape: torch.Size([5])
labels shape: torch.Size([5, 732])


In [39]:
args = TrainingArguments(
    output_dir="Amh-Llama-2",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="steps",
    eval_steps=5_000,
    logging_steps=5_000,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    weight_decay=0.1,
    warmup_steps=1_000,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    save_steps=5_000,
    fp16=True,
    push_to_hub=False,
)

In [40]:
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

In [41]:
trainer.train()

<IPython.core.display.Javascript object>